In [1]:
import subprocess
import pandas as pd
import os
import glob
from tqdm import tqdm
from os.path import basename as bn, join, split as sp
import parselmouth
import numpy as np
from parselmouth.praat import call
from scipy.io.wavfile import write
import praat_formants_python as pfp
import librosa
import matplotlib.pyplot as plt

In [2]:
# Write paths
ALL_EXP_FOLDER = "./exports/"
(lambda fp : os.mkdir(fp) if not os.path.exists(fp) else 0)(ALL_EXP_FOLDER) #make export folder

# Vowel subset Export CSV filename
SYNTH_VOWEL_FORMANT_ESTIMATION_IMP_FILENAME = "e_synth-vowels_formant_estimation_vowlimLIM.csv"
SYNTH_VOWEL_FORMANT_ESTIMATION_IMP_FILEPATH = join(ALL_EXP_FOLDER, SYNTH_VOWEL_FORMANT_ESTIMATION_IMP_FILENAME)

# TMP Audio Export folder
TEMP_AUDIO_EXP_FOLDER = "./audio_exports"
(lambda fp : os.mkdir(fp) if not os.path.exists(fp) else 0)(TEMP_AUDIO_EXP_FOLDER) #make export folder

# Noisy vowel export
NOISE_SYNTH_VOWEL_EXP_AUDIO_DATA_FOLDERNAME = "tempnoise5/"
NOISE_SYNTH_VOWEL_EXP_AUDIO_DATA_FOLDERPATH = join(TEMP_AUDIO_EXP_FOLDER, NOISE_SYNTH_VOWEL_EXP_AUDIO_DATA_FOLDERNAME)
(lambda fp : os.mkdir(fp) if not os.path.exists(fp) else 0)(NOISE_SYNTH_VOWEL_EXP_AUDIO_DATA_FOLDERPATH) #make export folder


0

In [3]:
VOWEL_LIMIT = 100
SYNTH_VOWELS_FORMANT_DF = pd.read_csv(SYNTH_VOWEL_FORMANT_ESTIMATION_IMP_FILEPATH.replace("LIM", f"{VOWEL_LIMIT}"))
SYNTH_VOWELS_FORMANT_DF

,index,person_id,sex,duration_second,vowel_name,pitch_mean_praat_base,F1_mean_praat_base,F2_mean_praat_base,F3_mean_praat_base,F1_median_praat_base,...,F1_mean_synthvow_praat,F2_mean_synthvow_praat,F3_mean_synthvow_praat,F1_median_synthvow_praat,F2_median_synthvow_praat,F3_median_synthvow_praat,F1_mean_synthvow_deepformant,F2_mean_synthvow_deepformant,F3_mean_synthvow_deepformant,F4_mean_synthvow_deepformant
0,0,MWGR0,M,0.066062,iy,118.56,495.03,2131.18,2950.80,495.03,...,477.77,964.17,2129.27,477.73,968.30,2129.67,476.76,1877.08,2744.39,4065.85
1,1,MJES0,M,0.074375,iy,112.66,316.93,2098.74,2844.58,325.92,...,326.33,895.63,2099.49,326.27,897.32,2094.84,407.13,1800.39,2574.27,3895.27
2,2,MSTK0,M,0.077688,iy,150.65,426.11,1931.06,2447.48,420.31,...,440.83,902.78,1938.58,440.39,910.59,1939.50,444.01,1756.68,2477.15,3927.64
3,3,MJDM0,M,0.121500,iy,94.78,467.55,1824.06,2588.08,450.22,...,460.02,846.94,1839.25,469.01,866.50,1820.84,456.66,1677.85,2489.65,3919.14
4,4,MLBC0,M,0.086125,iy,107.83,460.67,1942.34,2558.01,461.12,...,445.25,933.36,1942.31,445.38,942.06,1942.74,447.07,1746.88,2513.99,3947.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816,3816,MRJM3,M,0.085500,ax-h,112.14,743.40,1492.04,2672.56,671.51,...,741.56,912.03,1495.84,744.00,915.82,1494.80,584.48,1505.16,2527.48,3919.01
3817,3817,MAPV0,M,0.070187,ax-h,133.61,646.22,1847.19,2720.36,613.69,...,652.14,859.65,1851.73,655.59,869.90,1852.97,549.35,1770.26,2609.22,3974.69
3818,3818,MSFV0,M,0.174937,ax-h,99.52,765.19,1876.51,3195.29,600.50,...,762.64,892.74,1880.78,766.19,888.09,1881.88,636.02,1860.75,2884.74,4033.48
3819,3819,FLJG0,F,0.073313,ax-h,195.82,852.51,1813.99,2963.22,861.90,...,802.38,1082.95,1798.05,802.04,1086.73,1797.11,637.38,1798.50,2779.89,4045.44


In [7]:
import numpy as np
import scipy.signal as ss
import soundfile as sf
import rir_generator as rir
import shutil
from sklearn.utils import shuffle


In [8]:
def agaussn_vowel(aud_path):
    signal, fs = sf.read(aud_path)

    noise = 0.01 * np.random.normal(0, 1, len(signal))

    signal = 
    signal += noise
    fname = os.path.basename(aud_path)
    fname = f"agn-{fname}"
    shutil.copy(aud_path, NOISE_SYNTH_VOWEL_EXP_AUDIO_DATA_FOLDERPATH)
    dst_path = os.path.join(NOISE_SYNTH_VOWEL_EXP_AUDIO_DATA_FOLDERPATH, fname)
    
    sf.write(dst_path, signal, fs)

syvinfo = SYNTH_VOWELS_FORMANT_DF.loc[np.random.randint(0, len(SYNTH_VOWELS_FORMANT_DF))]

agaussn_vowel(syvinfo["synth_vowel_path"])

In [9]:
def reverb_vowel(aud_path):
    signal, fs = sf.read(aud_path, always_2d=True)

    h = rir.generate(
        c=340,                  # Sound velocity (m/s)
        fs=fs,                  # Sample frequency (samples/s)
        r=[                     # Receiver position(s) [x y z] (m)
            [2, 1.5, 1],
        ],
        s=[2, 3.5, 2],          # Source position [x y z] (m)
        L=[5, 4, 6],            # Room dimensions [x y z] (m)
        reverberation_time=0.4, # Reverberation time (s)
        nsample=4096,           # Number of output samples
    )

    # Convolve 2-channel signal with 3 impulse responses
    signal = ss.convolve(h[:, None, :], signal[:, :, None])
    signal = signal.T.flatten()
    signal = (signal - np.mean(signal)) / np.std(signal)

    fname = os.path.basename(aud_path)
    fname = f"rvb-{fname}"
    
    shutil.copy(aud_path, NOISE_SYNTH_VOWEL_EXP_AUDIO_DATA_FOLDERPATH)
    sf.write(os.path.join(NOISE_SYNTH_VOWEL_EXP_AUDIO_DATA_FOLDERPATH, fname), signal, fs)
    return signal, fs



syvinfo = SYNTH_VOWELS_FORMANT_DF.loc[np.random.randint(0, len(SYNTH_VOWELS_FORMANT_DF))]


reverb_vowel(syvinfo["synth_vowel_path"])

(array([0.00063202, 0.00063202, 0.00063202, ..., 0.00063202, 0.00063202,
        0.00063202]),
 16000)

In [20]:

filt_vowel = lambda s: SYNTH_VOWELS_FORMANT_DF["vowel_name"].str.contains("ah|ih|uh")
vowel_df = shuffle(SYNTH_VOWELS_FORMANT_DF.loc[filt_vowel], random_state=666)
vowel_df = vowel_df[:10]
vowel_df

,index,person_id,sex,duration_second,vowel_name,pitch_mean_praat_base,F1_mean_praat_base,F2_mean_praat_base,F3_mean_praat_base,F1_median_praat_base,...,F1_mean_synthvow_praat,F2_mean_synthvow_praat,F3_mean_synthvow_praat,F1_median_synthvow_praat,F2_median_synthvow_praat,F3_median_synthvow_praat,F1_mean_synthvow_deepformant,F2_mean_synthvow_deepformant,F3_mean_synthvow_deepformant,F4_mean_synthvow_deepformant
13687,13687,MRTC0,M,0.201437,ah,147.69,628.48,1585.99,2562.59,630.36,...,596.79,1019.66,1606.19,596.78,1019.84,1607.28,506.88,1548.96,2475.90,3960.88
13967,13967,FGWR0,F,0.096562,ah,190.93,642.64,1559.15,2870.95,666.04,...,592.35,1107.52,1552.74,592.20,1109.47,1552.00,514.20,1556.48,2663.33,4034.74
15370,15370,FCAJ0,F,0.103375,uh,170.32,589.29,1156.16,2917.03,589.18,...,557.24,991.40,1175.52,556.79,991.32,1174.76,502.62,1371.77,2604.23,3987.12
13533,13533,MTRR0,M,0.105438,ah,128.27,657.87,1408.13,2353.15,639.06,...,644.89,967.50,1413.22,644.53,963.60,1412.52,541.31,1464.12,2423.42,3913.37
14067,14067,FLAS0,F,0.105313,ah,168.16,835.79,1857.72,2634.47,845.97,...,827.44,894.40,1859.29,837.48,881.90,1852.75,664.47,1826.00,2599.77,3935.49
14087,14087,FVMH0,F,0.108375,ah,203.39,783.52,1828.46,2837.48,791.19,...,796.87,860.24,1830.89,799.66,862.26,1828.49,632.96,1834.71,2745.04,4097.60
4720,4720,MJXL0,M,0.140000,ih,121.43,568.04,1824.13,2651.98,574.13,...,571.83,819.92,1823.49,576.93,831.53,1824.42,511.63,1715.64,2534.38,3874.46
13433,13433,MJMP0,M,0.095562,ah,153.70,601.90,1583.90,2362.72,617.27,...,598.29,888.83,1599.86,605.30,880.71,1569.23,515.88,1552.48,2388.10,3931.94
5384,5384,FLJA0,F,0.201250,ih,222.29,558.83,2029.43,2885.82,568.47,...,571.05,824.69,2006.80,570.71,823.23,2006.35,533.36,1875.04,2715.56,4010.49
5205,5205,FRLL0,F,0.154375,ih,258.95,572.86,1853.67,2815.52,594.81,...,530.54,1107.44,1827.70,530.52,1111.11,1825.35,498.10,1752.51,2697.06,4032.00


In [21]:
for id, vowel in vowel_df.iterrows():

    agaussn_vowel(vowel["synth_vowel_path"])
    reverb_vowel(vowel["synth_vowel_path"])


In [ ]:
ey_15143_FJRP1_F_191

In [22]:
import scipy.io
mat = scipy.io.loadmat('/home/jeevan/Jeevan_K/Projects/Asquire/Vowtiar-Quest/vowtiar-formant_estimation/matlab/exports/ey_15143_FJRP1_F_191.mat')


In [25]:
print(mat["peak_freqs"])



[[ 200.1953125   391.6015625   551.7578125  1142.578125   1508.30078125
  1927.734375   2113.28125    2405.2734375  2684.08203125 2834.47265625
  2999.51171875 3462.40234375 3884.27734375 4382.32421875 4733.3984375
  5082.51953125 5303.22265625 6193.84765625 6744.140625   7291.015625
  7999.51171875]]
